# Investigation of "No Show" Appointments Dataset

<strong><i>By: Olamilekan Adenusi</i></strong>

*May 28, 2022*
## Table of Contents
<ul>
    <li><a href='#intro'>Introduction</a></li>
    <li><a href='#wrangling'>Data Wrangling</a></li>
    <li><a href='#eda'>Exploratory Data Analysis</a></li>
    <li><a href='#conclusions'>Conclusions</a></li>
</ul>

<a id='intro'></a>
# Introduction

In partial fulfillment of the requirements for passing the ALX-T Udacity Data Analyst course, this project has been undertaken. The aim of this project is to dive into a specific dataset with questions hoping to be answered by the end of the analysis. 

Medical appointments are being scheduled everyday but how do we determine those who show up for their appointments? Are there certain factors in this data that determine whether a patient shows up or not? For instance, are males more likely to miss their appointments than females? Does the appointment day have an effect on who shows up or not? Are older patients more committed to their medical schedules than younger ones? Are older patients at more risk of having a specific health problem? Let the data speak for itself! 

The dataset used in this analysis contains information about over 100,000 medical appointments in Brazil, focusing on whether patients **showed up or not** for their appointments. 

## Packages
The packages used in this analysis include:
1. <strong>pandas</strong> to read in the data/for data manipulation
2. <strong>numpy</strong> for working with arrays
3. <strong>matplotlib</strong> and <strong>seaborn</strong> for data visualization

These packages are now loaded below. 

In [1]:
# Render visualizations in the notebook
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use('seaborn')

<a id='wrangling'></a>
# Data Wrangling
This part is broken into three sections, namely, **data import** where the data is loaded and previewed and **data cleaning** where the data will be cleaned/trimmed for analysis. 

## I. Data Import and Preview
In this section, we will load the data and preview it to get a picture of what we are dealing with. Originally, this dataset can be obtained from [kaggle](https://www.google.com/url?q=https://www.kaggle.com/joniarroba/noshowappointments&sa=D&ust=1532469042118000). In this work, however it was directly downloaded via this [link](https://www.google.com/url?q=https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd2e9a_noshowappointments-kagglev2-may-2016/noshowappointments-kagglev2-may-2016.csv&sa=D&ust=1532469042118000). We will first load the data into a dataframe `df` using `pandas`. Then we will display summary about the data.

Although the column names are explanatory, it helps to still provide context. Below is some data description:
- **PatientId** and **AppointmentID** identify the patient and the appointment respectively. 
- **ScheduledDay** is the day someone made an appointment while **AppointmentDay** is the day of actual appointment.
- The columns **Scholarship, Hipertension, Alcoholism,  Diabetes, and SMS_received** all represent boolean values.
- **No-show** is a categorical column indicating whether a patient showed up (No) or not (Yes). This might be a bit weird to process at first, so in this work, the column shall be renamed to **Show** and its values changed so that Yes indicates a patient showed up while No indicates they didn't. 

In [2]:
# Load csv file
df = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')
print('There are {} instances and {} fields in the data'.format(*df.shape))

There are 110527 instances and 14 fields in the data


In [3]:
# Display summary of data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


- The data consists of different types such as integer, object, and float. 
- The "PatientId" and "AppointmentID" columns have incorrectly been treated as float and integers respectively where they ought to be strings. 
- The "ScheduledDay" and "AppointmentDay" would be better treated as date objects rather than mere strings.
- "Hipertension" and "Handcap" are incorrectly named 

In [ ]:
# display first 5 rows
df.head()

In [ ]:
# Display summary statistics for numeric columns
df.describe()

### Some Key Observations
- There are no missing values. 
- The Age column contains one or more negative values that must be handled appropriately. This will be investigated further in the next section.

## II. Data Cleaning
Some of the operations performed in this section are outlined below:
- To avoid having to look up the column names too often, we shall adopt a consistent column naming convention. I make all column names to be lower case letters.
- Perform the necessary data type casting for the relevant columns: **appointmentid** (int to str), **patientid** (float to str), **scheduledday** (object to datetime), and **appointmentday** (object to datetime). Because the patientid column is float type, it leaves a period that must be replaced by an empty string, e.g 2345678.0 to 2345678
- Rename "hipertension" to "hypertension" and "handcap" to "handicap" 
- Also rename "no-show" to "show" and convert the values to 1s and 0s so that 1 indicates Yes while 0 indicates No (a missed appointment)

In [4]:
# Rename columns to all lower case
df.columns = df.columns.str.lower()

In [5]:
# Change data types 
df['patientid'] = df['patientid'].astype('str').str.replace('.0', '', regex=False)
df['appointmentid'] = df['appointmentid'].astype('str')
df['scheduledday'] = pd.to_datetime(df['scheduledday'])
df['appointmentday'] = pd.to_datetime(df['appointmentday'])

In [6]:
# column renaming
df.rename(columns={'hipertension': 'hypertension', 'handcap':'handicap'}, inplace=True)

In [7]:
# column renaming
df.rename(columns={'no-show':'show'}, inplace=True)

# 1 --> patient showed up for the appointment, 0 --> patient missed the appointment
df['show'] = df['show'].apply(lambda x: 1 if x=='No' else 0)

### Examining patients with ages less than or equal to zero

In [ ]:
# Output data for patients with negative or zero age
df.query('age <= 0').head()

In [ ]:
# Get number of patients with ages <= 0
print('{} patients'.format(df.query('age <= 0').shape[0]))

In [ ]:
df.query('age <= 0').age.value_counts()

- There are 3540 patients with ages <= 0. Only **1** patient had a negative age. This is most likely an error in data collection. This data will be dropped. 
- Patients with zero age are most likely babies below 1 year. Let's check whether babies were diagnosed with alcoholism, hypertension, and/or diabetes. This would be weird since they are only babies. If there are none, then it is safe to conclude that they are indeed babies.

In [ ]:
# First drop the instance with a negative age
df.drop(df.query('age < 0').index, axis='rows', inplace=True)

Next extract those patients with zero age and perform necessary operations on the relevant columns.

In [ ]:
babies = df.query('age == 0')

In [ ]:
# first sum hypertension, alcoholism, and diabetes
babies[['hypertension', 'alcoholism', 'diabetes']].sum(axis=1)

In [ ]:
# then sum all the rows
babies[['hypertension', 'alcoholism', 'diabetes']].sum(axis=1).sum(axis=0)

Since there is no patient (with age = 0) that was diagnosed with alcoholism, diabetes, or hypertension, it is safe to say that patients with zero ages were all babies below 1 year. 

### Checking for Duplicate values

The next cell reveals that no two rows are exactly the same. 

In [ ]:
# Check for duplicate values
sum(df.duplicated())

But what about patient ids that are duplicated? In the dataframe below, the first 2 rows, for example, represent the same patient who had different medical appointments at Maria Ortiz and on different days. 

In [ ]:
df[ df.duplicated(subset='patientid', keep=False)].sort_values(by='patientid').head(2)

I would not be dropping these rows right away as they might be useful later when exploring the research questions. 

<a id='eda'></a>
# Exploratory Data Analysis

Let's first plot a histogram for all numeric columns in the data to get an idea of their distribution.

In [ ]:
plt.figure(figsize=(14, 14))

df.drop(['scheduledday', 'appointmentday'], axis='columns').hist()
plt.tight_layout();

## Research Question I
> How does the proportion of those who showed up for their appointments compare with those who did not? 

To answer this we need to make use of the **show** column and examine the number of patients associated with both outcomes.

In [ ]:
# Get number of patients for each unique value
df['show'].value_counts()

We can also visualize this with the aid of a pie chart that displays the proportion of patients.

In [ ]:
labels = ['Showed up', 'Didn\'t show up']

In [ ]:
plt.pie(df.show.value_counts(),
        labels=labels,
        explode=(0.2, 0.05),
        startangle=20,
        shadow=True,
        autopct='%.1f%%',
        textprops={'fontsize': 14})

plt.title('Percentage of patients who showed up and who didn\'t', fontsize=13);

- Thus, about **80%** of patients showed up for their appointments. Keep in mind that this includes patients who had more than one appointment. What if we were to narrow this down to patients that appear only once to be scheduled for an appointment? Would there be any change? 

To do this, we will first drop the data for every patient that had more than one appointment. This is done using pandas' `drop_duplicates` method. We will not be keeping any of the duplicates. For instance, in the dataframe below we will drop the two rows relating to the patient with id 111124532532143.

In [ ]:
df[ df.duplicated(subset='patientid', keep=False)].sort_values(by='patientid').head()

In [ ]:
df_unique_patients = df.drop_duplicates(subset='patientid', keep=False)

In [ ]:
plt.pie(df_unique_patients.show.value_counts(),
        labels=labels,
        explode=(0.2, 0.05),
        startangle=20,
        shadow=True,
        autopct='%.1f%%',
        textprops={'fontsize': 14})

plt.title('Percentage of patients who showed up and who didn\'t', fontsize=13);

- Again, just about **81%** of these patients show up for their medical appointments. This is not very different from the previous value. 

## Research Question II
> Are Females more committed to their medical appointment than males?

To explore this, let's make a countplot of the number of patients by gender.

In [ ]:
sns.catplot(x='gender', kind='count', data=df_unique_patients,  order=['M', 'F'])

# Set ylabel and title for plot
plt.ylabel('Number of patients')
plt.title('Number of Female and Male Patients', fontsize=13);

The above chart reveals that there are more female patients than male patients. But this does not prove that females are more committed. At best, it only tells us more females were scheduled for appointment than males. In the next chart we shall examine the proportion of females who showed up and compare this with the proportion of males who showed up to see if there is a difference. 

By grouping by the gender, the distribution of patients across **show** can be determined.

In [ ]:
df.groupby(by='gender')['show'].value_counts()

We however want the the values 0 and 1 to be in separate columns. To do this, the `unstack` method is used. Then a row operation is performed to convert the values to proportions using `apply`. 

In [ ]:
df.groupby(by='gender')['show'].value_counts().unstack()

In [ ]:
round(df.groupby(by='gender')['show'].value_counts().unstack().apply(lambda row: row/row.sum(), axis=1), 3)*100

In [ ]:
# Group the data by gender, unstack the MultiIndex, and apply a row operation
show_by_gender = (df.groupby(by='gender')['show'].value_counts()
                    .unstack()
                    .apply(lambda row: row/row.sum(), axis=1))

In [ ]:
# Plot the bar chart and add a y-axis label
show_by_gender.plot(kind='bar', stacked=True, rot=False, alpha=0.8)
plt.ylabel('Proportion of Patients')
plt.title('Proportion of Patients who showed up and didn\'t by gender')

# Adjust the legend
plt.legend(bbox_to_anchor=(1., 0.9), loc='center left', title='show', ncol=2);

The chart above shows that although there are more female than there are male patients, the proportion of females (**79.7%**) and males(**80%**) who showed up are very similar. Thus, it is safe to say that a patient's gender contributes very little to whether they will show up or not.

## Research Question III
> Are patients more likely to miss their appointments if their appointment dates are far?

Consider two patients, A and B. A's appointment day is in two days time, and B's appointment day is in 2 weeks time. Is B less likely to show up for their appointment given that their appointment day is farther?

Relevant columns, namely, `scheduledday` and `appointmentday` have been parsed as datetime objects to aid my analysis. The difference between these dates is computed and stored in the `days_away` column. But before proceeding, let's examine the two columns we'll be using further. At surface level, it seems the  `scheduledday` column has time component while the `appointmentday` column does not. 

In [ ]:
df[['scheduledday', 'appointmentday']].describe(datetime_is_numeric=True)

The output in the next cell is 0  which indicates the `appointmentday` column does not have time components. This must mean that the hospitals take only the day of appointment into account, and disregard the time.

In [ ]:
sum(df['appointmentday'].dt.hour)

Likewise, there is no schedule day that has a zero time component which means the hospital records not just the day, but also the time which a schedule was made. 

In [ ]:
df[ (df['scheduledday'].dt.hour == 0) & (df['scheduledday'].dt.minute ==0)]

Before we proceed we will need to `normalize` all schedule dates to midnight so that the time components are zeroed out just like `appointmentday`

In [ ]:
# zero the time components in the scheduledday column
df['scheduledday'] = df['scheduledday'].dt.normalize()

In [ ]:
# Then determine the number of days between appointment day and the schedule day
df['days_away'] = (df['appointmentday'] - df['scheduledday']).dt.days

In [ ]:
# Compute summary statistics on the new column
df['days_away'].describe()

The cell above reveals some of the days are negative, which is unexpected. Digging further, we check the number of occurrences of these instances. Output from the following cell shows that **4** patients had appointment a day before they made a schedule, while **1** patient had appointment 6 days before they were scheduled. I will be dropping these rows since there are only 5 occurrences.

**Note** that it is possible the appointment day was mistaken for the scheduled day in these cases. However, one can't be too sure. 

In [ ]:
df.query('days_away < 0').days_away.value_counts()

In [ ]:
# Filter for negative days
df.query('days_away < 0').head()

In [ ]:
# Drop instances with negative days
df.drop( df.query('days_away < 0').index, axis='rows', inplace=True)

Make a boxplot to show how the **days_away** is distributed based on the outcome of the appointment.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5)) #create a figure and axis object

# make a boxplot with some customizations. Ignore outliers by setting sym to an empty string
sns.boxplot(x='show',
            y='days_away',
            sym='',
            data=df,
            ax=ax, 
            order=[1, 0]);

ax.set_xticklabels(['Yes', 'No']) #Set tick labels for the x-axis
plt.title('Distribution of Number of days away by appointment outcome');

In [ ]:
df.groupby(by='show')['days_away'].median()

The plot clearly shows us that the median `days_away` for the patients who showed up for their appointments is much smaller than that for those who missed their appointments. In other words, 50% of those who showed up had appointment within at most 2 days of making a schedule, while 50% of those who did not show up had appointment within at most 11 days.

An empirical cumulative distribution plot (ecdf) might help us visualize this even much better. A function `plot_ecdf` is created below  for reuse. 

In [ ]:
def plot_ecdf(series, label=None, color='blue'):
    """
    Plot ecdf of a distribution
    ---------------------------
    Args: 
        series: a pandas series or a 1D array
    """
    
    x = np.sort(series)
    y = np.arange(1, len(x)+1) / len(x)
    
    plt.plot(x, y, marker='.', linestyle='none', markersize=2, label=label, color=color)
    
    # set plot title
    plt.title('ECDF plot for {}'.format(series.name))

In [ ]:
# plot ecdf for patients who showed up
plot_ecdf( df.query('show == 1')['days_away'], label='show', color='red')

# plot ecdf for patients who didn't show up
plot_ecdf(df.query('show == 0')['days_away'], label='no show', color='blue')

plt.xlabel('Days away')
plt.ylabel('probability')
plt.legend(prop={'size':14}, labelcolor=['red', 'blue'])

- The blue points are shifted to the right more than the red points, which indicates those who did not show up tend to have farther appointment dates than those who showed up.

## Research Question IV
> Does Age affect show up? 

Let's first get a feel for how the patients' ages are distributed using a histogram. Bins are created for every 10-year range. 

In [ ]:
bins = np.arange(0, 120, 10)

In [ ]:
df.age.plot(kind='hist', bins=bins, rwidth=0.96,  xlabel='Age', ylabel='Frequency', title='Distribution of Patient Age');

Let us now create age ranges and labels that we can work with. 

In [ ]:
bins = [-1, 12, 17, 24, 34, 44, 54, 64, 120]
labels = ['<=12', '13-17', '18-24', '25-34', '35-44', '45-54', '55-64', '>=65']

Using `pd.cut` allows us to separate the age column into age categories defined by the bins. 

In [ ]:
df['age_range'] = pd.cut(df.age, bins=bins, labels=labels)

In [ ]:
df[['age', 'age_range']].head()

In [ ]:
# set figure size
plt.figure(figsize=(8, 4))

# customize bar chart
df.groupby(by='age_range')['show'].mean().plot(kind='bar',
                                               ylabel='Mean show',
                                               title='Rate of show across age categories',
                                               rot=False,
                                               ylim=(0.2, 0.9));

Older patients within the age range 45 and above seem to be most committed to their medical appointments. Patients below age 13 are also more committed than those between 13-17 and 18-24. 

If we drill down for each gender, we notice a gradual increase in the show rate from patients aged 13 and above as we move up the age range for both genders.

In [ ]:
df.pivot_table(index='gender', columns='age_range', values='show')

## Research question V
> Does hypertension seem to be more common among older patients? 

To explore this we create a new dataframe that holds record of only one appointment for every unique patient. The number of unique hypertensive patients is preserved. 

In [ ]:
hyp = df.drop_duplicates(subset='patientid').query('hypertension == 1')

The mean age of hypertensive patients is 61 years. 

In [ ]:
hyp.age.mean()

Majority of those with hypertension are within the age range of 50-70 years as shown in the histogram below. 

In [ ]:
bins = np.arange(0, 120, 10)

In [ ]:
hyp.age.plot(kind='hist',
             rwidth=0.97,
             bins=bins,
             xlabel='Age',
             ylabel='Frequency',
             title='Age distribution for hypertensive patients');

In [ ]:
# plot ecdf for age of hypertensive patients
plot_ecdf(hyp.age)

plt.xlabel('age')
plt.ylabel('probability');

- The ages are almost normally distributed, but not quite. 
- The mean age of hypertensive patients is 61.

## Research question VI
> Are older patients more at risk of being hypertensive and diabetic at the same time?

Create a new dataframe for patients with hypertension and diabetes. Then obtain the normalized respective counts for each age range. 

In [ ]:
hyp_diab = hyp.query('diabetes == 1').age_range.value_counts(normalize=True)

In [ ]:
hyp_diab

In [ ]:
hyp_diab.plot(kind='bar', rot=False, 
              xlabel='Age range', ylabel='Proportion', 
              title='Proportion of hypertensive and diabetic patients by age range');

- Out of all patients with hypertension and diabetes, *48%* of them are at least 65 years of age, *30%* of them are between 55 and 64 years of age, and *15%* are between 45 and 54 years of age. The visualization shows a continuous decrease with decreasing age. 


<a id='conclusions'></a>
## Conclusions

1. **79.7%** of females and **80%** of males showed up for their appointments. This is a little difference and suggests that trying to predict whether a patient will show up or not based solely on their gender would not be so good. 
2. The date of appointment, as counted from the schedule date, influences the rate of show in the sense that the longer patients have to wait for their appointment, the less likely they are to show up. 
3. Older patients within the age range 45 and above seem to be most committed to their medical appointments. Patients below age 13 are also more committed than those between 13-17 and 18-24. If we drill down for each gender, we notice a gradual increase in the show rate from patients aged 13 and above as we move up the age range for both genders.
4. Hypertension is most common amongst older patients, particularly those within the age range of 50-70 years of age. Furthermore, majority of the patients with hypertension and diabetes happen to be older patients. While this does not imply a relationship between the two variables, it does suggest that older patients are at more risk of having these health problems.

## Limitations

- The reason for scheduling an appointment was not provided, thus making it hard to provide more context into  what makes some patients miss their appointments. 

## References

- [Medical Appointments No Show](https://www.kaggle.com/datasets/joniarroba/noshowappointments) - data source